In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score, accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv')

In [3]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [4]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [5]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [6]:
df.shape

(1462, 9)

In [7]:
df_cat = df.select_dtypes("object")
cat_list = list(df_cat.columns)
df_num = df.select_dtypes(["int64","float64"])
num_list = list(df_num)

In [8]:
df_cat.shape, df_num.shape

((1462, 4), (1462, 5))

In [9]:
df[cat_list]= df[cat_list].fillna("NA")
df[num_list]= df[num_list].fillna(0)

In [10]:
df_cat['industry'].mode()[0]

'retail'

In [11]:
df_num.corrwith(df['lead_score']).sort_values()

number_of_courses_viewed   -0.004879
annual_income               0.005334
interaction_count           0.009888
converted                   0.193673
lead_score                  1.000000
dtype: float64

In [12]:
df_num.corrwith(df['number_of_courses_viewed']).sort_values()

interaction_count          -0.023565
lead_score                 -0.004879
annual_income               0.031551
converted                   0.435914
number_of_courses_viewed    1.000000
dtype: float64

In [13]:
df_num.corrwith(df['interaction_count']).sort_values()

number_of_courses_viewed   -0.023565
lead_score                  0.009888
annual_income               0.048618
converted                   0.374573
interaction_count           1.000000
dtype: float64

In [14]:
# annual_income and interaction_count is the answer
df_num.corrwith(df['annual_income']).sort_values()

number_of_courses_viewed    0.009770
lead_score                  0.015610
interaction_count           0.027036
converted                   0.053131
annual_income               1.000000
dtype: float64

In [15]:
y = df['converted']
df = df.drop('converted', axis= 1)

In [16]:
np.random.seed(42)
full_data, df_test, full_y, y_test = train_test_split(df, y, test_size =.2)
df_train, df_val, y_train, y_val = train_test_split(full_data, full_y, test_size =.2)

In [17]:
for i in cat_list:
    x= mutual_info_score(df_train[i],y_train)
    print(i,round(x,2))

lead_source 0.03
industry 0.01
employment_status 0.01
location 0.0


In [18]:
dv = DictVectorizer(sparse= False)

In [19]:
train_dict = df_train.to_dict(orient='records') 
X_train = dv.fit_transform(train_dict)

In [20]:
val_dict = df_val.to_dict(orient="records")
X_val = dv.transform(val_dict)

In [21]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [22]:
y_pred = model.predict(X_val)

In [23]:
accuracy_score(y_val,y_pred)

0.7606837606837606

In [24]:
# the answer is industry
scores = []

for i in df_train.columns:
    train = df_train.drop(i, axis=1)
    train_dict = train.to_dict(orient='records') 
    X_train = dv.fit_transform(train_dict)
    val_dict = df_val.to_dict(orient="records")
    X_val = dv.transform(val_dict)
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    score = accuracy_score(y_val,y_pred)
    scores.append(score)
    print(i,score)
    print(0.7606837606837606 - score)
    print()

lead_source 0.7564102564102564
0.004273504273504258

industry 0.7564102564102564
0.004273504273504258

number_of_courses_viewed 0.6623931623931624
0.09829059829059827

annual_income 0.8333333333333334
-0.07264957264957272

employment_status 0.7478632478632479
0.012820512820512775

location 0.7606837606837606
0.0

interaction_count 0.7136752136752137
0.04700854700854695

lead_score 0.7692307692307693
-0.008547008547008628



In [25]:
df_train

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score
993,paid_ads,retail,2,54023.0,self_employed,middle_east,2,0.70
212,events,NA,1,72608.0,student,middle_east,3,0.40
563,social_media,healthcare,0,77320.0,employed,asia,1,0.35
871,paid_ads,retail,0,13929.0,unemployed,asia,1,0.69
1111,NA,other,1,44308.0,self_employed,NA,3,0.11
...,...,...,...,...,...,...,...,...
236,paid_ads,retail,2,0.0,self_employed,africa,1,0.09
1115,referral,manufacturing,1,93897.0,employed,south_america,2,0.35
290,paid_ads,retail,1,46393.0,unemployed,south_america,4,0.70
957,NA,education,3,89042.0,employed,asia,4,0.75


In [29]:
for c in [0.01, 0.1, 1, 10, 100]:
    train_dict = full_data.to_dict(orient='records') 
    X_train = dv.fit_transform(train_dict)
    test_dict = df_test.to_dict(orient="records")
    X_test = dv.transform(test_dict)
    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    model.fit(X_train, full_y)
    y_pred = model.predict(X_test)
    score = accuracy_score(y_test,y_pred)
    print(c)
    print(score)

0.01
0.7337883959044369
0.1
0.7372013651877133
1
0.7372013651877133
10
0.7372013651877133
100
0.7372013651877133


# NOTE: all the o/ps were the same that's why i used the test set 